In [91]:
from collections import defaultdict
from fastprogress import progress_bar, master_bar
import json
from pathlib import Path
import pandas as pd
import scispacy
import spacy
from tqdm import tqdm, tqdm_notebook
import vaex

tqdm.pandas()

In [2]:
#Data Paths
root_path = Path.cwd().parents[0]
processed_dir = root_path / "processed"

Load the processed data. 

In [136]:
df = pd.read_parquet(processed_dir / "documents_processed.parquet")

In [137]:
df.head()

,abstract,body_text,sha,doi_x,doi_y,source_x,journal,has_full_text
0,Bordetella bronchiseptica isolate KM22 has bee...,20-kb insert library preparation protocol (htt...,8f8eb4f004c2002face0723f2f58cc411954d36e,http://dx.doi.org/10.1128/MRA.01207-19,http://dx.doi.org/10.1128/MRA.01207-19,PMC,Microbiol Resour Announc,True
1,,I nfectious diseases have been an ever-present...,63f7049d200896290b38b38711113054f7ea1b50,http://dx.doi.org/10.3201/eid1104.041167,http://dx.doi.org/10.3201/eid1104.041167,PMC,Emerg Infect Dis,True
2,The influenza A nucleoprotein (NP) is an attra...,The transmission of a pathogenic avian H5N1 vi...,4df45b8404d9de0b376a8ae3c282a517df36fe51,http://dx.doi.org/10.1051/vetres/2009071,http://dx.doi.org/10.1051/vetres/2009071,PMC,Vet Res,True
3,The outbreak of severe acute respiratory syndr...,acute lung injury; inflammatory response; neut...,e0737ee93afe7b0bf06b1e3f9adf21d541dd10f0,http://dx.doi.org/10.1038/labinvest.2012.92,http://dx.doi.org/10.1038/labinvest.2012.92,PMC,Lab Invest,True
4,The prevalence of feline herpesvirus-1 (FHV-1)...,Feline herpesvirus type 1 (FHV-1) is the most ...,3c3572ba243d61e7631725669c8f88347fdbd5bc,http://dx.doi.org/10.4142/jvs.2008.9.2.207,http://dx.doi.org/10.4142/jvs.2008.9.2.207,PMC,J Vet Sci,True


In [29]:
df.shape

(13259, 8)

Load the pretrained NER SciSpacy models. These models identify different things, so we will use most of them. The "bc5cdr" only has labels Chemical and Disease. The other models are more fine-grained.

In [43]:
bionlp = spacy.load('en_ner_bionlp13cg_md')
jnlpba = spacy.load('en_ner_jnlpba_md')
craft = spacy.load('en_ner_craft_md')

### View a sample of what each model identifies

In [41]:
text_sample = df.iloc[2]['abstract']
print(text_sample)

The influenza A nucleoprotein (NP) is an attractive target for avian flu vaccine development because of its high conversancy in the evolutionary chain of the virus. Here we identified two novel HLA-A*0201 restricted NP epitopes, named H5N1 NP373-381 AMDSNTLEL (NP373) and NP458-466 FQGRGVFEL (NP458), using computational bioinformatic analysis. The NP peptides showed a high binding affinity to HLA-A*0201 on T2 cells, and were able to induce the activation of the cytotoxic T cells in the human peripheral blood mononuclear cells. We examined the potential of using NP373 and NP458 peptide sequences supplemented with a single-chain trimer as potential DNA vaccine candidates in an HHD transgenic mouse model. A gene gun delivery system was used for administrating the vaccine candidates into the animals. The results from cytotoxicity and ELISPOT assays indicated that a significant amount of IFN-c was secreted by the T cells of the vaccinated mice, and the T cells were able to eliminate the corr

In [46]:
models = {"bionlp": bionlp, "jnlpba": jnlpba, "craft": craft}

In [47]:
for key, model in models.items():    
    doc = model(text_sample)
    ents = list(doc.ents)
    print(key)
    for ent in ents:
        print(f"{ent.label_}: {ent.text}")
    print("\n")

bionlp
ORGANISM: influenza A nucleoprotein
GENE_OR_GENE_PRODUCT: NP
GENE_OR_GENE_PRODUCT: HLA-A*0201
GENE_OR_GENE_PRODUCT: NP epitopes
ORGANISM: H5N1 NP373-381 AMDSNTLEL
SIMPLE_CHEMICAL: NP458-466 FQGRGVFEL
SIMPLE_CHEMICAL: NP458
GENE_OR_GENE_PRODUCT: NP peptides
GENE_OR_GENE_PRODUCT: HLA-A*0201
CELL: T2 cells
CELL: cytotoxic T cells
ORGANISM: human peripheral blood mononuclear cells
CELLULAR_COMPONENT: DNA
ORGANISM: mouse
CELL: T cells
ORGANISM: mice
CELL: T cells
CELL: peptide-loaded T2 cells
GENE_OR_GENE_PRODUCT: NP peptides
ORGANISM: H5N1 nucleoprotein /
GENE_OR_GENE_PRODUCT: single-chain
ORGANISM: mouse
GENE_OR_GENE_PRODUCT: boyxie@ust.hk 1


jnlpba
PROTEIN: influenza A nucleoprotein
PROTEIN: NP
PROTEIN: HLA-A*0201 restricted NP epitopes
DNA: H5N1 NP373-381 AMDSNTLEL (NP373) and NP458-466 FQGRGVFEL
PROTEIN: NP458
PROTEIN: HLA-A*0201
CELL_LINE: T2 cells
CELL_TYPE: cytotoxic T cells
CELL_TYPE: human peripheral blood mononuclear cells
DNA: NP373 and NP458 peptide sequences
PROTEIN: s

### Run the models on the full text

In [50]:
docs = df['abstract'].progress_apply(models['craft'])

100%|██████████| 13259/13259 [06:20<00:00, 34.82it/s]


In [67]:
entities = []
for doc in tqdm_notebook(docs):
    doc_entities = defaultdict(set)
    for ent in list(doc.ents):
        doc_entities[ent.label_].add(ent.text)
    entities.append(doc_entities)
entities

/Users/akdidier/virtualenvironments/cord/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[defaultdict(set, {'TAXON': {'swine'}, 'SO': {'genome sequence'}}),
 defaultdict(set, {}),
 defaultdict(set,
             {'TAXON': {'animals', 'avian', 'human', 'mice', 'virus'},
              'SO': {'DNA',
               'epitope',
               'epitopes',
               'evolutionary chain of the virus',
               'gene',
               'peptide sequences',
               'peptides',
               'transgenic mouse'},
              'CL': {'T cells', 'blood mononuclear cells', 'cells'},
              'GGP': {'NP373'},
              'CHEBI': {'peptide-loaded'}}),
 defaultdict(set,
             {'TAXON': {'Coronavirus murine',
               'animal',
               'mice',
               'viral',
               'virus'},
              'GGP': {'MHV', 'PTX3', 'pentraxin PTX3'},
              'SO': {'wild type', 'wild type mice'},
              'CL': {'leukocyte',
               'macrophages',
               'neutrophil',
               'neutrophils'}}),
 defaultdict(set,
       

In [68]:
entity_df = pd.DataFrame(entities)

In [69]:
entity_df.head()

,TAXON,SO,CL,GGP,CHEBI,GO
0,{swine},{genome sequence},NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,"{human, mice, virus, animals, avian}","{peptide sequences, epitopes, epitope, transge...","{blood mononuclear cells, T cells, cells}",{NP373},{peptide-loaded},NaN
3,"{mice, animal, virus, Coronavirus murine, viral}","{wild type mice, wild type}","{leukocyte, macrophages, neutrophils, neutrophil}","{MHV, pentraxin PTX3, PTX3}",NaN,NaN
4,"{calicivirus, FHV-1, C. felis, FCV, C., animal...",NaN,NaN,NaN,NaN,NaN


In [64]:
new_df = df.join(entity_df)

In [65]:
new_df.head()

,abstract,body_text,sha,doi_x,doi_y,source_x,journal,has_full_text,TAXON,SO,CL,GGP,CHEBI,GO
0,Bordetella bronchiseptica isolate KM22 has bee...,20-kb insert library preparation protocol (htt...,8f8eb4f004c2002face0723f2f58cc411954d36e,http://dx.doi.org/10.1128/MRA.01207-19,http://dx.doi.org/10.1128/MRA.01207-19,PMC,Microbiol Resour Announc,True,[swine],"[genome sequence, genome sequence]",NaN,NaN,NaN,NaN
1,,I nfectious diseases have been an ever-present...,63f7049d200896290b38b38711113054f7ea1b50,http://dx.doi.org/10.3201/eid1104.041167,http://dx.doi.org/10.3201/eid1104.041167,PMC,Emerg Infect Dis,True,NaN,NaN,NaN,NaN,NaN,NaN
2,The influenza A nucleoprotein (NP) is an attra...,The transmission of a pathogenic avian H5N1 vi...,4df45b8404d9de0b376a8ae3c282a517df36fe51,http://dx.doi.org/10.1051/vetres/2009071,http://dx.doi.org/10.1051/vetres/2009071,PMC,Vet Res,True,"[avian, human, animals, mice, avian, virus]","[evolutionary chain of the virus, epitopes, pe...","[cells, T cells, blood mononuclear cells, T ce...",[NP373],[peptide-loaded],NaN
3,The outbreak of severe acute respiratory syndr...,acute lung injury; inflammatory response; neut...,e0737ee93afe7b0bf06b1e3f9adf21d541dd10f0,http://dx.doi.org/10.1038/labinvest.2012.92,http://dx.doi.org/10.1038/labinvest.2012.92,PMC,Lab Invest,True,"[viral, Coronavirus murine, virus, animal, mic...","[wild type, wild type mice]","[leukocyte, neutrophils, macrophages, neutrophil]","[pentraxin PTX3, PTX3, PTX3, PTX3, PTX3, MHV, ...",NaN,NaN
4,The prevalence of feline herpesvirus-1 (FHV-1)...,Feline herpesvirus type 1 (FHV-1) is the most ...,3c3572ba243d61e7631725669c8f88347fdbd5bc,http://dx.doi.org/10.4142/jvs.2008.9.2.207,http://dx.doi.org/10.4142/jvs.2008.9.2.207,PMC,J Vet Sci,True,"[herpesvirus-1, FHV-1, calicivirus, FCV, C., f...",NaN,NaN,NaN,NaN,NaN


In [138]:
for name, model in models.items():
    print(name)
    docs = df['abstract'].progress_apply(model)
    entities = []
    for doc in progress_bar(docs, parent=mb):
        doc_entities = defaultdict(set)
        for ent in list(doc.ents):
            doc_entities[ent.label_].add(ent.text)
        for key, val in doc_entities.items():
            doc_entities[key] = list(val)
        entities.append(doc_entities)
    entity_df = pd.DataFrame(entities)
    print(f"entity_df columns: {entity_df.columns}")
    df = df.join(entity_df)





  0%|          | 0/13259 [00:00<?, ?it/s]

bionlp






  0%|          | 2/13259 [00:00<22:00, 10.04it/s]



  0%|          | 5/13259 [00:00<18:17, 12.08it/s]



  0%|          | 10/13259 [00:00<14:24, 15.32it/s]



  0%|          | 16/13259 [00:00<11:20, 19.46it/s]



  0%|          | 24/13259 [00:00<08:51, 24.90it/s]



  0%|          | 31/13259 [00:00<07:28, 29.51it/s]



  0%|          | 37/13259 [00:00<06:40, 32.97it/s]



  0%|          | 42/13259 [00:01<06:27, 34.13it/s]



  0%|          | 47/13259 [00:01<06:03, 36.34it/s]



  0%|          | 52/13259 [00:01<05:48, 37.91it/s]



  0%|          | 57/13259 [00:01<05:31, 39.82it/s]



  0%|          | 65/13259 [00:01<04:50, 45.43it/s]



  1%|          | 71/13259 [00:01<04:46, 46.06it/s]



  1%|          | 78/13259 [00:01<04:39, 47.21it/s]



  1%|          | 88/13259 [00:01<03:57, 55.52it/s]



  1%|          | 97/13259 [00:02<03:33, 61.79it/s]



  1%|          | 104/13259 [00:02<03:33, 61.58it/s]



  1%|          | 111/13259 [00:02<04:00, 54.78it/s]



  1%|          | 120/132

  6%|▌         | 812/13259 [00:16<03:26, 60.34it/s]



  6%|▌         | 819/13259 [00:17<03:22, 61.53it/s]



  6%|▌         | 826/13259 [00:17<04:01, 51.46it/s]



  6%|▋         | 838/13259 [00:17<03:20, 61.91it/s]



  6%|▋         | 846/13259 [00:17<04:48, 43.08it/s]



  6%|▋         | 853/13259 [00:17<04:31, 45.73it/s]



  7%|▋         | 862/13259 [00:17<03:55, 52.58it/s]



  7%|▋         | 871/13259 [00:18<03:39, 56.32it/s]



  7%|▋         | 878/13259 [00:18<03:31, 58.45it/s]



  7%|▋         | 885/13259 [00:18<03:54, 52.76it/s]



  7%|▋         | 891/13259 [00:18<04:23, 47.00it/s]



  7%|▋         | 899/13259 [00:18<03:58, 51.91it/s]



  7%|▋         | 905/13259 [00:18<03:51, 53.42it/s]



  7%|▋         | 911/13259 [00:18<03:47, 54.39it/s]



  7%|▋         | 923/13259 [00:18<03:30, 58.67it/s]



  7%|▋         | 930/13259 [00:19<03:31, 58.17it/s]



  7%|▋         | 937/13259 [00:19<03:52, 53.11it/s]



  7%|▋         | 943/13259 [00:19<04:18, 47.68it/s]



  7%|▋    

 13%|█▎        | 1677/13259 [00:33<03:41, 52.39it/s]



 13%|█▎        | 1683/13259 [00:33<03:50, 50.25it/s]



 13%|█▎        | 1692/13259 [00:33<03:21, 57.32it/s]



 13%|█▎        | 1699/13259 [00:33<03:46, 51.09it/s]



 13%|█▎        | 1709/13259 [00:33<03:18, 58.06it/s]



 13%|█▎        | 1716/13259 [00:33<04:03, 47.37it/s]



 13%|█▎        | 1722/13259 [00:34<03:57, 48.56it/s]



 13%|█▎        | 1728/13259 [00:34<04:19, 44.39it/s]



 13%|█▎        | 1733/13259 [00:34<04:30, 42.63it/s]



 13%|█▎        | 1738/13259 [00:34<04:31, 42.38it/s]



 13%|█▎        | 1743/13259 [00:34<04:35, 41.80it/s]



 13%|█▎        | 1749/13259 [00:34<04:23, 43.65it/s]



 13%|█▎        | 1754/13259 [00:34<04:54, 39.07it/s]



 13%|█▎        | 1761/13259 [00:34<04:16, 44.87it/s]



 13%|█▎        | 1766/13259 [00:35<04:46, 40.17it/s]



 13%|█▎        | 1773/13259 [00:35<04:17, 44.67it/s]



 13%|█▎        | 1778/13259 [00:35<05:34, 34.33it/s]



 13%|█▎        | 1783/13259 [00:35<05:42, 33.48i

...

 92%|█████████▏| 12172/13259 [13:11<01:24, 12.83it/s]



 92%|█████████▏| 12174/13259 [13:11<01:25, 12.63it/s]



 92%|█████████▏| 12177/13259 [13:11<01:19, 13.59it/s]



 92%|█████████▏| 12180/13259 [13:11<01:10, 15.28it/s]



 92%|█████████▏| 12182/13259 [13:12<01:09, 15.48it/s]



 92%|█████████▏| 12187/13259 [13:12<01:00, 17.78it/s]



 92%|█████████▏| 12189/13259 [13:12<01:00, 17.62it/s]



 92%|█████████▏| 12192/13259 [13:12<00:55, 19.22it/s]



 92%|█████████▏| 12197/13259 [13:12<00:51, 20.76it/s]



 92%|█████████▏| 12200/13259 [13:13<01:07, 15.73it/s]



 92%|█████████▏| 12203/13259 [13:13<00:58, 17.98it/s]



 92%|█████████▏| 12206/13259 [13:13<00:59, 17.81it/s]



 92%|█████████▏| 12208/13259 [13:13<01:07, 15.46it/s]



 92%|█████████▏| 12210/13259 [13:13<01:14, 14.11it/s]



 92%|█████████▏| 12213/13259 [13:13<01:06, 15.78it/s]



 92%|█████████▏| 12215/13259 [13:13<01:08, 15.30it/s]



 92%|█████████▏| 12217/13259 [13:14<01:08, 15.21it/s]



 92%|█████████▏| 12219/13259 [1

 94%|█████████▍| 12462/13259 [13:32<03:06,  4.28it/s]



 94%|█████████▍| 12466/13259 [13:32<02:21,  5.61it/s]



 94%|█████████▍| 12469/13259 [13:33<01:53,  6.93it/s]



 94%|█████████▍| 12472/13259 [13:33<01:29,  8.80it/s]



 94%|█████████▍| 12474/13259 [13:33<01:18,  9.99it/s]



 94%|█████████▍| 12476/13259 [13:33<01:11, 11.03it/s]



 94%|█████████▍| 12478/13259 [13:33<01:07, 11.57it/s]



 94%|█████████▍| 12480/13259 [13:33<01:10, 11.10it/s]



 94%|█████████▍| 12482/13259 [13:33<01:02, 12.34it/s]



 94%|█████████▍| 12485/13259 [13:34<00:51, 14.98it/s]



 94%|█████████▍| 12487/13259 [13:34<00:58, 13.30it/s]



 94%|█████████▍| 12491/13259 [13:34<00:46, 16.60it/s]



 94%|█████████▍| 12494/13259 [13:34<00:46, 16.37it/s]



 94%|█████████▍| 12497/13259 [13:34<00:47, 16.08it/s]



 94%|█████████▍| 12499/13259 [13:35<01:06, 11.45it/s]



 94%|█████████▍| 12501/13259 [13:35<00:59, 12.72it/s]



 94%|█████████▍| 12503/13259 [13:35<00:54, 13.94it/s]



 94%|█████████▍| 12506/13259 [1

 96%|█████████▌| 12753/13259 [13:52<00:23, 21.68it/s]



 96%|█████████▌| 12756/13259 [13:52<00:28, 17.69it/s]



 96%|█████████▌| 12758/13259 [13:52<00:29, 17.20it/s]



 96%|█████████▌| 12760/13259 [13:52<00:28, 17.82it/s]



 96%|█████████▋| 12762/13259 [13:52<00:32, 15.18it/s]



 96%|█████████▋| 12766/13259 [13:53<00:26, 18.57it/s]



 96%|█████████▋| 12769/13259 [13:53<00:28, 17.17it/s]



 96%|█████████▋| 12772/13259 [13:53<00:33, 14.57it/s]



 96%|█████████▋| 12774/13259 [13:53<00:31, 15.33it/s]



 96%|█████████▋| 12776/13259 [13:53<00:33, 14.30it/s]



 96%|█████████▋| 12779/13259 [13:54<00:30, 15.65it/s]



 96%|█████████▋| 12782/13259 [13:54<00:28, 16.80it/s]



 96%|█████████▋| 12784/13259 [13:54<00:27, 17.48it/s]



 96%|█████████▋| 12787/13259 [13:54<00:25, 18.18it/s]



 96%|█████████▋| 12790/13259 [13:54<00:25, 18.21it/s]



 96%|█████████▋| 12792/13259 [13:54<00:26, 17.76it/s]



 96%|█████████▋| 12794/13259 [13:55<00:46,  9.90it/s]



 97%|█████████▋| 12798/13259 [1

 98%|█████████▊| 13039/13259 [14:14<00:15, 14.57it/s]



 98%|█████████▊| 13042/13259 [14:14<00:14, 14.78it/s]



 98%|█████████▊| 13044/13259 [14:14<00:15, 14.00it/s]



 98%|█████████▊| 13046/13259 [14:14<00:15, 13.62it/s]



 98%|█████████▊| 13048/13259 [14:14<00:15, 13.44it/s]



 98%|█████████▊| 13051/13259 [14:14<00:13, 14.99it/s]



 98%|█████████▊| 13053/13259 [14:15<00:15, 13.47it/s]



 98%|█████████▊| 13055/13259 [14:15<00:14, 13.83it/s]



 98%|█████████▊| 13057/13259 [14:15<00:15, 13.38it/s]



 98%|█████████▊| 13059/13259 [14:15<00:16, 12.09it/s]



 99%|█████████▊| 13063/13259 [14:15<00:13, 14.62it/s]



 99%|█████████▊| 13065/13259 [14:15<00:15, 12.29it/s]



 99%|█████████▊| 13067/13259 [14:16<00:17, 10.75it/s]



 99%|█████████▊| 13069/13259 [14:16<00:18, 10.03it/s]



 99%|█████████▊| 13071/13259 [14:16<00:21,  8.79it/s]



 99%|█████████▊| 13073/13259 [14:16<00:21,  8.62it/s]



 99%|█████████▊| 13075/13259 [14:17<00:18, 10.08it/s]



 99%|█████████▊| 13077/13259 [1

entity_df columns: Index(['TAXON', 'SO', 'CL', 'GGP', 'CHEBI', 'GO'], dtype='object')


In [139]:
df.head()

,abstract,body_text,sha,doi_x,doi_y,source_x,journal,has_full_text,ORGANISM,ORGANISM_SUBSTANCE,...,PROTEIN,CELL_LINE,CELL_TYPE,RNA,TAXON,SO,CL,GGP,CHEBI,GO
0,Bordetella bronchiseptica isolate KM22 has bee...,20-kb insert library preparation protocol (htt...,8f8eb4f004c2002face0723f2f58cc411954d36e,http://dx.doi.org/10.1128/MRA.01207-19,http://dx.doi.org/10.1128/MRA.01207-19,PMC,Microbiol Resour Announc,True,[Bordetella bronchiseptica],[swine],...,[KM22],NaN,NaN,NaN,[swine],[genome sequence],NaN,NaN,NaN,NaN
1,,I nfectious diseases have been an ever-present...,63f7049d200896290b38b38711113054f7ea1b50,http://dx.doi.org/10.3201/eid1104.041167,http://dx.doi.org/10.3201/eid1104.041167,PMC,Emerg Infect Dis,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The influenza A nucleoprotein (NP) is an attra...,The transmission of a pathogenic avian H5N1 vi...,4df45b8404d9de0b376a8ae3c282a517df36fe51,http://dx.doi.org/10.1051/vetres/2009071,http://dx.doi.org/10.1051/vetres/2009071,PMC,Vet Res,True,"[H5N1 NP373-381 AMDSNTLEL, mouse, mice, influe...",NaN,...,"[NP, single-chain trimer, HLA-A*0201, HLA-A*02...","[T2 cells, peptide-loaded T2 cells]","[cytotoxic T cells, T cells, human peripheral ...",NaN,"[human, mice, virus, animals, avian]","[peptide sequences, epitopes, epitope, transge...","[blood mononuclear cells, T cells, cells]",[NP373],[peptide-loaded],NaN
3,The outbreak of severe acute respiratory syndr...,acute lung injury; inflammatory response; neut...,e0737ee93afe7b0bf06b1e3f9adf21d541dd10f0,http://dx.doi.org/10.1038/labinvest.2012.92,http://dx.doi.org/10.1038/labinvest.2012.92,PMC,Lab Invest,True,"[Coronavirus murine hepatitis virus strain 1, ...",[serum],...,"[SARS, MHV -1, Long pentraxin PTX3, Cytokine, ...",NaN,"[macrophages, neutrophils]",NaN,"[mice, animal, virus, Coronavirus murine, viral]","[wild type mice, wild type]","[leukocyte, macrophages, neutrophils, neutrophil]","[MHV, pentraxin PTX3, PTX3]",NaN,NaN
4,The prevalence of feline herpesvirus-1 (FHV-1)...,Feline herpesvirus type 1 (FHV-1) is the most ...,3c3572ba243d61e7631725669c8f88347fdbd5bc,http://dx.doi.org/10.4142/jvs.2008.9.2.207,http://dx.doi.org/10.4142/jvs.2008.9.2.207,PMC,J Vet Sci,True,"[feline calicivirus, FHV-1, C. felis, FCV, cat...",NaN,...,NaN,NaN,NaN,NaN,"[calicivirus, FHV-1, C. felis, FCV, C., animal...",NaN,NaN,NaN,NaN,NaN


In [140]:
df.columns

Index(['abstract', 'body_text', 'sha', 'doi_x', 'doi_y', 'source_x', 'journal',
       'has_full_text', 'ORGANISM', 'ORGANISM_SUBSTANCE', 'CELL',
       'SIMPLE_CHEMICAL', 'GENE_OR_GENE_PRODUCT', 'CELLULAR_COMPONENT',
       'ORGAN', 'CANCER', 'MULTI-TISSUE_STRUCTURE', 'TISSUE',
       'PATHOLOGICAL_FORMATION', 'AMINO_ACID', 'ORGANISM_SUBDIVISION',
       'IMMATERIAL_ANATOMICAL_ENTITY', 'ANATOMICAL_SYSTEM',
       'DEVELOPING_ANATOMICAL_STRUCTURE', 'DNA', 'PROTEIN', 'CELL_LINE',
       'CELL_TYPE', 'RNA', 'TAXON', 'SO', 'CL', 'GGP', 'CHEBI', 'GO'],
      dtype='object')

In [141]:
extracted_data = df.to_dict(orient='records')

In [129]:
def as_list(tup):
    obj = tup[0]
    key = tup[1]
    #throws an error if the field is nan, but we just want to keep these as is
    try:
        obj[key] = list(obj[key])
        return obj
    except TypeError:
        return obj

In [132]:
keys_to_map = ['TAXON_craft', 'SO_craft', 'CL_craft',
       'GGP_craft', 'CHEBI_craft', 'GO_craft']

for key in progress_bar(keys_to_map):
    arg = list(zip(extracted_data, [key]*len(extracted_data)))
    extracted_data = list(map(as_list, arg))

In [144]:
extracted_data[0]

{'abstract': 'Bordetella bronchiseptica isolate KM22 has been used in experimental infections of swine as a model of clinical B. bronchiseptica infection and to study host-to-host transmission. The draft genome sequence of KM22 was reported in 2014. Here, we report the complete genome sequence of KM22.',
 'body_text': "20-kb insert library preparation protocol (https://www.pacb.com/wp-content/uploads/ Procedure-Checklist-20-kb-Template-Preparation-Using-BluePippin-Size-Selection -System.pdf). The 20-kb library was sequenced with a PacBio RS II platform using two single-molecule real-time (SMRT) cells, resulting in 283,436 total reads and an average read length of 7,600 bp. Reads were subsequently assessed for quality using FastQC (https://www.bioinformatics.babraham.ac.uk/projects/fastqc/).\n Whole-genome assemblies were generated using PacBio SMRT Analysis v. 2.3.0 and Canu v. 1.5 software. The average PacBio coverage for the assembled genome was 475ϫ. Assembling the PacBio data resul

In [143]:
with open(processed_dir / "entities_extracted_abstract.json", "w") as f:
    json.dump(extracted_data, f)